<h1 style='font-weight:bold'> Tutorial </h1>

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from sklearn.datasets import load_iris
import pandas as pd

In [10]:
import os

os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"

In [11]:
# 1. Create SparkSession
spark = SparkSession.builder.appName("IrisClassification").getOrCreate()

In [12]:
# 2. Load Iris dataset using pandas
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target
spark_df = spark.createDataFrame(df)

In [13]:
# 3. Encode label column
indexer = StringIndexer(inputCol="label", outputCol="indexedLabel")
spark_df = indexer.fit(spark_df).transform(spark_df)

Py4JJavaError: An error occurred while calling o138.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 2.0 failed 1 times, most recent failure: Lost task 7.0 in stage 2.0 (TID 31) (DESKTOP-48OVVTK executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD

In [ ]:
# 4. Assemble features
assembler = VectorAssembler(inputCols=iris.feature_names, outputCol="features")
spark_df = assembler.transform(spark_df)

In [ ]:
# 5. Split into train/test
train_df, test_df = spark_df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# 6. Choose classifier (Random Forest)
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="features")

In [ ]:
# 7. Cross-validation & grid search
paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [5, 10, 20]).build()
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", metricName="accuracy")
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

In [ ]:
# 8. Train the model
cv_model = cv.fit(train_df)

In [ ]:
# 9. Predict & Evaluate
predictions = cv_model.transform(test_df)
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.4f}")
predictions.select("prediction", "indexedLabel").show(5)

Setup

In [5]:
import findspark
findspark.init('/usr/hdp/current/spark-client')  # this is Spark path in HDP

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ConnectToSandbox") \
    .master("yarn") \
    .config("spark.hadoop.yarn.resourcemanager.hostname", "sandbox-hdp.hortonworks.com") \
    .config("spark.yarn.access.hadoopFileSystems", "hdfs://sandbox-hdp.hortonworks.com:8020") \
    .getOrCreate()

spark


The output above shows that

In [14]:
import os
from pyspark.sql import SparkSession

# Optional: if needed, point to your Python executable
# os.environ["PYSPARK_PYTHON"] = "python"

# Set the IP address from docker inspect
docker_ip = "172.17.0.2"  # <--- replace with actual IP

spark = SparkSession.builder \
    .appName("DockerSparkConnection") \
    .master("yarn") \
    .config("spark.hadoop.yarn.resourcemanager.address", f"{docker_ip}:8088") \
    .config("spark.hadoop.fs.defaultFS", f"hdfs://{docker_ip}:8020") \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()

spark


In [15]:
spark.range(5).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [16]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("HiveExample") \
    .master("yarn") \
    .enableHiveSupport() \
    .config("spark.hadoop.yarn.resourcemanager.address", "172.17.0.2:8088") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://172.17.0.2:8020") \
    .config("hive.metastore.uris", "thrift://172.17.0.2:9083") \
    .getOrCreate()


In [20]:
df = spark.sql("SHOW TABLES IN default")
df.show()


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf() \
    .setAppName("PrintSquaredRDD") \
    .setMaster("yarn") \
    .set("spark.hadoop.yarn.resourcemanager.address", "172.17.0.2:8088") \
    .set("spark.hadoop.fs.defaultFS", "hdfs://172.17.0.2:8020")

sc = SparkContext(conf=conf)

rdd = sc.parallelize([1, 2, 3, 4])
squaredRDD = rdd.map(lambda x: x * x)
result = squaredRDD.collect()

print("Squared RDD:", result)

sc.stop()



PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [2]:
conda create -n sparkenv python=3.9
conda activate sparkenv

SyntaxError: invalid syntax (2966563341.py, line 1)